# Evaluate trained model

In [1]:
%load_ext autoreload
%autoreload 1

In [38]:
import os
import cv2

import sys
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image


try:
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
    import labinet.object_detect
    import labinet.box
except ImportError:
    # This part is only required to run the notebook
    # directory when the module itself is not installed.
    #
    # If you have the module installed, just use "import labinet..."
    import os
    import inspect
    # the .travis.yml is coded so that we execute tests from within test subdir. Relative to 'test' the .py is found in ../source/dev
    cmd_subfolder = os.path.realpath(os.path.abspath(os.path.join(os.path.split(inspect.getfile(inspect.currentframe()))[0], "..")))
    if cmd_subfolder not in sys.path:
        sys.path.insert(0, cmd_subfolder)
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
    import labinet.object_detect
    import labinet.box

%aimport labinet.io_util
%aimport labinet.object_detect
%aimport labinet.box

%matplotlib inline

## change path, model name etc to your needs

In [3]:
MODEL_NAME = 'inference_graph'   # the result from Step 6 Export Inference Graph
IMAGE_PATH = 'images'
IMAGE_NAME = '01-20190414173244-01.jpg'   # cmdline arg
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
CWD_PATH = os.path.join(os.getcwd(),'..')  # should become gitbase
NUM_CLASSES = 1

In [4]:
# model
PATH_TO_MODEL = os.path.join(CWD_PATH, MODEL_NAME, 'frozen_inference_graph.pb')
# label map
LABEL_MAP = os.path.join(CWD_PATH, 'data', 'object-detection.pbtxt')
# path to image
PATH_TO_IMAGE = os.path.join(CWD_PATH, IMAGE_PATH, IMAGE_NAME)

## Load (frozen) Model into Memory

In [5]:
detection_graph = load_model(PATH_TO_MODEL)
#sess = tf.Session(graph=detection_graph)
   

### load labels 

In [6]:
categories, category_index = load_label_map(LABEL_MAP, NUM_CLASSES)

## Run Inference : Object Detection 
_load the image and run on it the object detection_

In [7]:
# load image
image =  Image.open(PATH_TO_IMAGE)
#imshow(np.asarray(image))

# inference
output_dict = labinet.object_detect.run_inference_for_single_image(image, detection_graph)

### Visualize result

In [13]:
image_with_boxes = labinet.object_detect.visualize_boxes_after_detection(image, output_dict, category_index)
# All the results have been drawn on image. Now display the image.
plt.figure(figsize=IMAGE_SIZE)
#plt.imshow(image_with_boxes)
print(IMAGE_NAME)


01-20190414173244-01.jpg


<Figure size 864x576 with 0 Axes>

## Write Result as XML
_use the xml file to laod image in labelImg and check correct boxing_

In [40]:
boxes = output_dict['detection_boxes']
scores = output_dict['detection_scores']
boxes_to_use = labinet.object_detect.get_boxes_to_use(boxes, scores)

In [17]:
# we need the box' coordinates relative to the image size
boxes_normed = []
for box in boxes_to_use:
    box_normed = labinet.box.get_normalized_coordinates(box, image.size)
    boxes_normed.append(box_normed)

In [54]:
%aimport labinet.box

xml = labinet.box.box_to_labelimg_xml(IMAGE_NAME, image.size, boxes_normed, imagepath=PATH_TO_IMAGE)
ET.dump(xml)
xmlfname = PATH_TO_IMAGE[:-4]+".xml"
xml.write(xmlfname)

<annotation><folder>images</folder><filename>01-20190414173244-01.jpg</filename><path>C:\Users\Carsten\Documents\python\object_detection\labinet\notebook\..\images\01-20190414173244-01.jpg</path><source><database>Unknown</database></source><size><width>480</width><height>640</height><depth>3</depth></size><segmented>0</segmented><object><name>dog</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>124</xmin><xmax>307</xmax><ymin>426</ymin><ymax>591</ymax></bndbox></object><object><name>dog</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>157</xmin><xmax>366</xmax><ymin>456</ymin><ymax>585</ymax></bndbox></object><object><name>dog</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>214</xmin><xmax>381</xmax><ymin>463</ymin><ymax>575</ymax></bndbox></object></annotation>


## Do inference an all new files


In [107]:
NEW_IMAGE_PATH = os.path.join(CWD_PATH, 'new_images')
files = os.listdir(NEW_IMAGE_PATH)

In [94]:
for fname in files:
    if fname.endswith('jpg'):
        image_file = os.path.join(NEW_IMAGE_PATH, fname)
        print(f'inference on: {fname}')
        image =  Image.open(image_file)
        output_dict = labinet.object_detect.run_inference_for_single_image(image, detection_graph)
        boxes = output_dict['detection_boxes']
        scores = output_dict['detection_scores']
        boxes_to_use = labinet.object_detect.get_boxes_to_use(boxes, scores)
        boxes_normed = []
        print(f'found {len(boxes_to_use)} objects')
        for box in boxes_to_use:
            box_normed = labinet.box.get_normalized_coordinates(box, image.size)
            boxes_normed.append(box_normed)
            print(box_normed)
        xml = labinet.box.box_to_labelimg_xml(fname, image.size, boxes_normed, imagepath=image_file)
        xmlfname = image_file[:-4]+".xml"
        #xml.write(xmlfname)
        break


inference on: 01-20190414173236-00.jpg
with graph as default
get tensor:num_detections:0
set: num_detections:0
get tensor:detection_boxes:0
set: detection_boxes:0
get tensor:detection_scores:0
set: detection_scores:0
get tensor:detection_classes:0
set: detection_classes:0
get tensor:detection_masks:0
get tensor:image_tensor:0
set: image_tensor:0
found 2 objects
{'xmin': 216, 'xmax': 367, 'ymin': 392, 'ymax': 513}
{'xmin': 421, 'xmax': 480, 'ymin': 452, 'ymax': 618}


```
inference on: 01-20190414173236-00.jpg
found 2 objects
{'xmin': 216, 'xmax': 367, 'ymin': 392, 'ymax': 513}
{'xmin': 421, 'xmax': 480, 'ymin': 452, 'ymax': 618}
```

In [96]:
import numpy as np
#print(image_np_exp.shape)
import tensorflow as tf
image_width=480
image_height=640
image_np_exp = np.expand_dims(labinet.io_util.load_image_into_numpy_array(image), axis=0) 
image_np_exp.shape  # 1, 640, 480, 3


(1, 640, 480, 3)

In [108]:
tensor_dict = labinet.object_detect.get_tensor_dict_with_masks(image_width, image_height, detection_graph)
image_tensor = tensor_dict['image_tensor']

with tf.Session(graph=detection_graph) as sess:
    for fname in files:
        if fname.endswith('jpg'):
            # load and converte image
            image_file = os.path.join(NEW_IMAGE_PATH, fname)
            print(f'inference on: {fname}')
            image =  Image.open(image_file)
            image_np_exp = np.expand_dims(labinet.io_util.load_image_into_numpy_array(image), axis=0) 
            
            # inference
            output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image_np_exp})
            labinet.object_detect.convert_output_dict(output_dict)

            # boxes to xml
            boxes = output_dict['detection_boxes']
            scores = output_dict['detection_scores']
            boxes_to_use = labinet.object_detect.get_boxes_to_use(boxes, scores)
            boxes_normed = []
            print(f'found {len(boxes_to_use)} objects')
            for box in boxes_to_use:
                box_normed = labinet.box.get_normalized_coordinates(box, image.size)
                boxes_normed.append(box_normed)
                print(box_normed)
            xml = labinet.box.box_to_labelimg_xml(fname, image.size, boxes_normed, imagepath=image_file)
            # xml to disk
            xmlfname = image_file[:-4]+".xml"
            xml.write(xmlfname)
            break


inference on: 01-20190414173201-00.jpg
found 0 objects
inference on: 01-20190414173202-00.jpg
found 0 objects
inference on: 01-20190414173205-00.jpg
found 1 objects
{'xmin': 419, 'xmax': 480, 'ymin': 405, 'ymax': 583}
inference on: 01-20190414173206-00.jpg
found 0 objects
inference on: 01-20190414173207-00.jpg
found 0 objects
inference on: 01-20190414173208-00.jpg
found 0 objects
inference on: 01-20190414173230-00.jpg
found 2 objects
{'xmin': 271, 'xmax': 394, 'ymin': 335, 'ymax': 436}
{'xmin': 341, 'xmax': 414, 'ymin': 386, 'ymax': 551}
inference on: 01-20190414173231-00.jpg
found 1 objects
{'xmin': 332, 'xmax': 462, 'ymin': 417, 'ymax': 539}
inference on: 01-20190414173232-00.jpg
found 1 objects
{'xmin': 260, 'xmax': 334, 'ymin': 343, 'ymax': 512}
inference on: 01-20190414173233-00.jpg
found 2 objects
{'xmin': 214, 'xmax': 333, 'ymin': 437, 'ymax': 636}
{'xmin': 327, 'xmax': 418, 'ymin': 402, 'ymax': 547}
inference on: 01-20190414173234-00.jpg
found 0 objects
inference on: 01-2019041

found 1 objects
{'xmin': 242, 'xmax': 439, 'ymin': 470, 'ymax': 559}
inference on: 01-20190414173634-00.jpg
found 1 objects
{'xmin': 242, 'xmax': 439, 'ymin': 469, 'ymax': 559}
inference on: 01-20190414173707-00.jpg
found 1 objects
{'xmin': 241, 'xmax': 437, 'ymin': 470, 'ymax': 559}
inference on: 01-20190414173709-00.jpg
found 1 objects
{'xmin': 243, 'xmax': 436, 'ymin': 471, 'ymax': 561}
inference on: 01-20190414173710-00.jpg
found 1 objects
{'xmin': 239, 'xmax': 429, 'ymin': 471, 'ymax': 556}
inference on: 01-20190414173712-00.jpg
found 1 objects
{'xmin': 237, 'xmax': 429, 'ymin': 472, 'ymax': 555}
inference on: 01-20190414173727-00.jpg
found 0 objects
inference on: 01-20190414173729-00.jpg
found 1 objects
{'xmin': 243, 'xmax': 429, 'ymin': 463, 'ymax': 563}
inference on: 01-20190414173730-00.jpg
found 1 objects
{'xmin': 257, 'xmax': 427, 'ymin': 459, 'ymax': 565}
inference on: 01-20190414173740-00.jpg
found 1 objects
{'xmin': 246, 'xmax': 462, 'ymin': 428, 'ymax': 550}
inference on

found 2 objects
{'xmin': 218, 'xmax': 477, 'ymin': 449, 'ymax': 613}
{'xmin': 33, 'xmax': 315, 'ymin': 508, 'ymax': 640}
inference on: 02-20190414174400-00.jpg
found 2 objects
{'xmin': 234, 'xmax': 462, 'ymin': 465, 'ymax': 630}
{'xmin': 49, 'xmax': 340, 'ymin': 503, 'ymax': 640}
inference on: 02-20190414174403-00.jpg
found 1 objects
{'xmin': 261, 'xmax': 438, 'ymin': 468, 'ymax': 613}
inference on: 02-20190414174405-00.jpg
found 1 objects
{'xmin': 354, 'xmax': 480, 'ymin': 425, 'ymax': 595}
inference on: 02-20190414174406-00.jpg
found 0 objects
inference on: 02-20190414174431-00.jpg
found 0 objects
inference on: 02-20190414174432-00.jpg
found 0 objects
inference on: 02-20190414174434-00.jpg
found 0 objects
inference on: 02-20190414174435-00.jpg
found 1 objects
{'xmin': 40, 'xmax': 371, 'ymin': 548, 'ymax': 639}
inference on: 02-20190414174436-00.jpg
found 1 objects
{'xmin': 15, 'xmax': 361, 'ymin': 548, 'ymax': 640}
inference on: 02-20190414174437-00.jpg
found 1 objects
{'xmin': 78, '

found 1 objects
{'xmin': 105, 'xmax': 282, 'ymin': 458, 'ymax': 640}
inference on: 02-20190414174700-00.jpg
found 1 objects
{'xmin': 15, 'xmax': 285, 'ymin': 478, 'ymax': 640}
inference on: 02-20190414174701-00.jpg
found 1 objects
{'xmin': 23, 'xmax': 209, 'ymin': 441, 'ymax': 640}
inference on: 02-20190414174702-00.jpg
found 1 objects
{'xmin': 0, 'xmax': 188, 'ymin': 437, 'ymax': 638}
inference on: 02-20190414174703-00.jpg
found 1 objects
{'xmin': 77, 'xmax': 302, 'ymin': 457, 'ymax': 637}
inference on: 02-20190414174704-00.jpg
found 1 objects
{'xmin': 70, 'xmax': 208, 'ymin': 411, 'ymax': 614}
inference on: 02-20190414174705-00.jpg
found 1 objects
{'xmin': 74, 'xmax': 333, 'ymin': 466, 'ymax': 606}
inference on: 02-20190414174706-00.jpg
found 2 objects
{'xmin': 174, 'xmax': 371, 'ymin': 448, 'ymax': 640}
{'xmin': 22, 'xmax': 347, 'ymin': 541, 'ymax': 640}
inference on: 02-20190414174707-00.jpg
found 2 objects
{'xmin': 275, 'xmax': 419, 'ymin': 473, 'ymax': 640}
{'xmin': 14, 'xmax': 2

found 0 objects
inference on: 02-20190414174811-00.jpg
found 0 objects
inference on: 02-20190414174812-00.jpg
found 0 objects
inference on: 02-20190414174813-00.jpg
found 0 objects
inference on: 02-20190414174814-00.jpg
found 1 objects
{'xmin': 261, 'xmax': 469, 'ymin': 422, 'ymax': 562}
inference on: 02-20190414174815-00.jpg
found 1 objects
{'xmin': 272, 'xmax': 471, 'ymin': 423, 'ymax': 566}
inference on: 02-20190414174816-00.jpg
found 1 objects
{'xmin': 272, 'xmax': 473, 'ymin': 423, 'ymax': 568}
inference on: 02-20190414174817-00.jpg
found 1 objects
{'xmin': 277, 'xmax': 474, 'ymin': 424, 'ymax': 569}
inference on: 02-20190414174818-00.jpg
found 1 objects
{'xmin': 276, 'xmax': 474, 'ymin': 424, 'ymax': 569}
inference on: 02-20190414174819-00.jpg
found 1 objects
{'xmin': 283, 'xmax': 475, 'ymin': 422, 'ymax': 567}
inference on: 02-20190414174820-00.jpg
found 1 objects
{'xmin': 278, 'xmax': 476, 'ymin': 423, 'ymax': 568}
inference on: 02-20190414174821-00.jpg
found 1 objects
{'xmin':

found 1 objects
{'xmin': 259, 'xmax': 390, 'ymin': 415, 'ymax': 536}
inference on: 02-20190414174933-00.jpg
found 1 objects
{'xmin': 223, 'xmax': 367, 'ymin': 415, 'ymax': 533}
inference on: 02-20190414174934-00.jpg
found 1 objects
{'xmin': 234, 'xmax': 372, 'ymin': 414, 'ymax': 530}
inference on: 02-20190414174935-00.jpg
found 1 objects
{'xmin': 221, 'xmax': 401, 'ymin': 414, 'ymax': 535}
inference on: 02-20190414174936-00.jpg
found 1 objects
{'xmin': 239, 'xmax': 422, 'ymin': 393, 'ymax': 541}
inference on: 02-20190414174937-00.jpg
found 1 objects
{'xmin': 262, 'xmax': 449, 'ymin': 407, 'ymax': 544}
inference on: 02-20190414174938-00.jpg
found 1 objects
{'xmin': 296, 'xmax': 473, 'ymin': 429, 'ymax': 547}
inference on: 02-20190414174939-00.jpg
found 1 objects
{'xmin': 311, 'xmax': 480, 'ymin': 406, 'ymax': 541}
inference on: 02-20190414174940-00.jpg
found 1 objects
{'xmin': 293, 'xmax': 480, 'ymin': 411, 'ymax': 544}
inference on: 02-20190414174941-00.jpg
found 1 objects
{'xmin': 305

found 1 objects
{'xmin': 286, 'xmax': 476, 'ymin': 461, 'ymax': 556}
inference on: 03-20190414175500-00.jpg
found 1 objects
{'xmin': 82, 'xmax': 336, 'ymin': 488, 'ymax': 640}
inference on: 03-20190414175503-00.jpg
found 0 objects
inference on: 03-20190414175505-00.jpg
found 0 objects


In [ ]:
# for everything like video stream or so... do 
# labinet.object_detect.get_tensor_dict_with_masks(...)
# with tf.Session(graph=detection_graph) as sess:
#   ....
#   output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image_np_exp})
#   ....

